In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
import pandas as pd
data = pd.read_csv('text_emotion.csv')

In [3]:
data = data[['sentiment', 'content']]
labels = data.sentiment 
texts = data.content
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [4]:
data.shape

(40000, 2)

In [6]:
import re
texts = texts.apply(lambda x: x.lower())
texts = texts.apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [7]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
texts=texts.apply(lambda sen:" ".join(x for x in sen.split() if x not in stop))
texts.head()

0    tiffanylue know listenin bad habit earlier sta...
1               layin n bed headache ughhhhwaitin call
2                        funeral ceremonygloomy friday
3                              wants hang friends soon
4    dannycastillo want trade someone houston ticke...
Name: content, dtype: object

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=30

In [9]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print(texts[:10])
Y = data['sentiment']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(Y)
Y=le.transform(Y) 
labels = to_categorical(np.asarray(Y))
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', labels.shape)

Found 53145 unique tokens.
0    tiffanylue know listenin bad habit earlier sta...
1               layin n bed headache ughhhhwaitin call
2                        funeral ceremonygloomy friday
3                              wants hang friends soon
4    dannycastillo want trade someone houston ticke...
5    repinging ghostridah14 didnt go prom bc bf did...
6    sleep im thinking old friend want hes married ...
7                                hmmm httpwwwdjherocom
8                         charviray charlene love miss
9                       kelcouch im sorry least friday
Name: content, dtype: object
Shape of data tensor: (40000, 30)
Shape of label tensor: (40000, 13)


In [10]:
from sklearn.model_selection import train_test_split


Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

x_val = X_train[:5000]
partial_x_train = X_train[5000:]
y_val = Y_train[:5000]
partial_y_train = Y_train[5000:]

(26800, 30) (26800, 13)
(13200, 30) (13200, 13)


In [11]:
from keras import models 
from keras import layers 
from keras.layers import Embedding
from keras.layers import Dense, Embedding, LSTM
from keras.models import Sequential
from keras.layers import Dense

embeddings_index = {}
f = open('glove.42B.300d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1917494 word vectors.


In [12]:
from keras import models 
from keras import layers 
from keras.layers import Embedding
from keras.layers import Dense, Embedding, LSTM
EMBEDDING_DIM = 300 
num_words = min(MAX_NB_WORDS, len(word_index)) 
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [13]:
embed_dim = 300
lstm_out = 196

model = Sequential()
model.add(embedding_layer)
model.add(layers.Dropout(0.5))
model.add(LSTM(196, dropout=0.25, recurrent_dropout=0.2))
model.add(layers.Dropout(0.5))
model.add(Dense(13,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 300)           6000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               389648    
_________________________________________________________________
dropout_2 (Dropout)          (None, 196)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 13)                2561      
Total params: 6,392,209
Trainable params: 392,209
Non-trainable params: 6,000,000
_____

In [17]:
model.fit(partial_x_train, partial_y_train,
          batch_size=128,
          epochs=15,
          validation_data=(x_val, y_val))

Train on 21800 samples, validate on 5000 samples
Epoch 1/15
21800/21800 [==============================] - 61s 3ms/step - loss: 1.8391 - acc: 0.3672 - val_loss: 1.8461 - val_acc: 0.3720
Epoch 2/15
21800/21800 [==============================] - 68s 3ms/step - loss: 1.8238 - acc: 0.3743 - val_loss: 1.8413 - val_acc: 0.3662
Epoch 3/15
21800/21800 [==============================] - 63s 3ms/step - loss: 1.8175 - acc: 0.3753 - val_loss: 1.8427 - val_acc: 0.3698
Epoch 4/15
21800/21800 [==============================] - 62s 3ms/step - loss: 1.8172 - acc: 0.3751 - val_loss: 1.8462 - val_acc: 0.3710
Epoch 5/15
21800/21800 [==============================] - 61s 3ms/step - loss: 1.8045 - acc: 0.3842 - val_loss: 1.8426 - val_acc: 0.3718
Epoch 6/15
21800/21800 [==============================] - 59s 3ms/step - loss: 1.8026 - acc: 0.3828 - val_loss: 1.8431 - val_acc: 0.3712
Epoch 7/15
21800/21800 [==============================] - 61s 3ms/step - loss: 1.7927 - acc: 0.3826 - val_loss: 1.8463 - val_acc:

In [18]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 1.87
acc: 0.37
